In [2]:
import pandas as pd
import altair as alt
# alt.data_transformers.enable("vegafusion")
alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

# Evaluate

In [3]:
df = pd.read_csv('../javascript/JAN-10-2024/gov_aggregated_results_only_image-alt.csv', header=None, names=['page_id', 'issue_id', 'violations', 'passes'])

# df = df[df.issue_id == 'image-alt']

df.head(10)
# len(df.issue_id.unique().tolist())

,page_id,issue_id,violations,passes
0,1,image-alt,0,6
1,12,image-alt,0,10
2,16,image-alt,0,1
3,15,image-alt,0,12
4,20,image-alt,0,3
5,19,image-alt,0,9
6,17,image-alt,0,14
7,2,image-alt,0,4
8,23,image-alt,0,19
9,14,image-alt,3,28


$$
A3 = 1 - \Pi_b (1 - F_b)^{\frac{B_pb}{N_pb} + \frac{B_pb}{B_p}}
$$

> Equation 3 presents the formula for computing the A3 metric, where Bpb is the total of actual points of failure of a checkpoint b in page p, b is the barrier (checkpoint violation), Npb is the total of potential points of failure of a checkpoint b in page p, and Fb identifies the severity of a certain barrier b (this weight is calculated by simple heuristics, by combining the results of an automatic evaluation and manual testing or by disabled users feedback [221). The authors of this metric performed an experimental study to compare the results between A3 and UWEM and understand the differences between them. A checkpoint weight of 0.05 was used for all checkpoints, assuming that all of them would have the same importance. This experiment was conducted with a group of six disabled users that evaluated six web pages. After applying both metrics, the authors concluded that A3 outperformed UWEM in the experiment [11].

In [18]:
_df = df.merge(df.groupby(['page_id'])['violations'].sum().reset_index().rename(columns={'violations':'Bp'}), on='page_id', how='left')

_df['Npb'] = _df['violations'] + _df['passes']
_df['Bpb_over_Npb'] = _df['violations'] / _df['Npb']
_df['Bpb_over_Bb'] = _df['violations'] / _df['Bp']
_df['Fb'] = 0.1

_df = _df.merge(_df.groupby(['page_id'])['Npb'].sum().reset_index().rename(columns={'Npb':'Np'}), on='page_id', how='left')

_df['A3'] = 1 - _df['Fb']
_df['A3'] = _df['A3'] ** (_df['Bpb_over_Npb'] + _df['Bpb_over_Bb'])
_df.drop(['issue_id'], inplace=True, axis=1)
_df = _df[['page_id', 'A3']].groupby(['page_id']).prod().reset_index()
_df['A3'] = 1 - _df['A3']

_df

,page_id,A3
0,0,0.190000
1,3,0.111775
2,9,0.115069
3,13,0.105907
4,14,0.124637


In [4]:
# or just ratio
_df = df.copy()

_df.drop(['issue_id'], axis=1, inplace=True)

_df = _df.groupby(['page_id']).sum().reset_index()

_df['Np'] = _df['violations'] + _df['passes']
_df['violation_ratio'] = _df['violations'] / _df['Np']
# _df['Bpb_over_Npb'] = _df['violations'] / _df['Npb']
# _df['Bpb_over_Bb'] = _df['violations'] / _df['Bp']
# _df['Fb'] = 0.1

# _df = _df.merge(_df.groupby(['page_id'])['Npb'].sum().reset_index().rename(columns={'Npb':'Np'}), on='page_id', how='left')

# _df['A3'] = 1 - _df['Fb']
# _df['A3'] = _df['A3'] ** (_df['Bpb_over_Npb'] + _df['Bpb_over_Bb'])
# _df.drop(['issue_id'], inplace=True, axis=1)
# _df = _df[['page_id', 'A3']].groupby(['page_id']).prod().reset_index()
# _df['A3'] = 1 - _df['A3']

_df

,page_id,violations,passes,Np,violation_ratio
0,1,0,6,6,0.000000
1,2,0,4,4,0.000000
2,3,1,7,8,0.125000
3,5,0,15,15,0.000000
4,6,0,13,13,0.000000
5,7,0,12,12,0.000000
6,8,0,9,9,0.000000
7,9,2,22,24,0.083333
8,11,0,7,7,0.000000
9,12,0,10,10,0.000000


In [6]:
_df.columns

Index(['page_id', 'violations', 'passes', 'Np', 'violation_ratio'], dtype='object')

In [7]:
alt.Chart(
    _df
).mark_tick(
    # opacity=0.01
).encode(
    alt.X(f'Np:Q'),
    # alt.Y('country:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)

In [13]:
alt.Chart(
    _df #[_df.country.isin(COUNTRIES)]
).mark_tick(
    # opacity=0.01
).encode(
    alt.X(f'violations:Q'),
    # alt.Y('country:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)

In [17]:
alt.Chart(
    _df #[_df.country.isin(COUNTRIES)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'violation_ratio:Q').axis(format='.0%').scale(alt.Scale(domain=[0, 1])),
    alt.Y('page_id:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)

In [19]:
alt.Chart(
    _df #[_df.country.isin(COUNTRIES)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'A3:Q'),
    alt.Y('page_id:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)